In [1]:
import numpy as np



In [8]:
w1 = 3
w2 = 5 
b = -2

x = w1 * 0.4 + w2 * -.6 + b
print(x)

print( 1. / (1 + np.exp(x)))

-3.8
0.9781187290638694


## Back Propagation ##
***
CS231n that has a simplified RNN that does not take any input x, and only computes the recurrence on the hidden state (equivalently, the input x could always be zero):

In [23]:
T = 50
H = 5

wHH = np.random.randn( H,H )

ss = {}
hs = {}
hs[-1] = np.random.randn(H)


for t in range(T):
    ss[t] = np.dot( wHH, hs[t-1] )
    hs[t] = np.maximum(0, ss[t])

print( hs[0].shape )
print( ss[0].shape )
    
dhs = {}
dss = {}
dhs[T-1] = np.random.randn( H,H )
# backpropagation
for t in reversed( range(T)):
    dss[t] = ( hs[t] > 0 ) * dhs[t]
    dhs[t-1] = np.dot( wHH.T, dss[t] )

    

(5,)
(5,)


In [24]:
x = [10,5]
np.sum(x)

15

# MiniFlow UdaCity #

In [109]:
class Node:
    def __init__(self, inbound_nodes=[]):
        # Nodes from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Nodes to which this Node passes values
        self.outbound_nodes = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented


class Input(Node):
    def __init__(self):
        # An Input Node has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Node.__init__(self)

        
        # NOTE: Input Node is the only Node where the value
        # may be passed as an argument to forward().
        #
        # All other Node implementations should get the value
        # of the previous nodes from self.inbound_nodes
        #
        # Example:
        # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        #print("input forward call.")
        #print(value)
        if value is not None:
            self.value = value


class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        print("Linear type inputs.",type(inputs))
        
        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        in_, w_, b_  = self.inbound_nodes
        print(in_.value)
        print(w_.value)
        print(b_.value)
        res = np.dot(in_.value, w_.value) + b_.value
        print(res)
        self.value = res
        

def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
                
    
    return L


def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value


In [110]:
#In general, there's no restriction on the values that can be passed to an Input node.
#from miniflow import *

inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3],
    weights: [0.5, 0.25, 1.4],
    bias: 2
}

graph = topological_sort(feed_dict)

output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

Linear type inputs. <class '__main__.Input'>
[6, 14, 3]
[0.5, 0.25, 1.4]
2
12.7
12.7
